<a href="https://colab.research.google.com/github/vvalcristina/analise_covid_19/blob/master/COVID_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Análise de Dados Covid-19**



Análise das séries temporais sobre a contaminação do COVID-19.
Utilizando o DataSet do [Kagle](https://www.kaggle.com/imdevskp/corona-virus-report).

In [0]:
#Importações de dados
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

In [0]:
#Importando o DataSet
df = pd.read_csv("covid_19_clean_complete.csv", parse_dates=['Date'])

In [3]:
#Verificando o DataFrame
df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0
1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0,0
2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0,0
3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0,0
4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0,0


**Pré-processamento dos dados**

Antes de iniciarmos as analises removeremos os Warnings do DataSet

In [0]:
import warnings
warnings.filterwarnings('ignore')

Antes de iniciarmos a análise existem 1 fator que devemos nos atentar, as colunas não podem ter caracteres especiais.


In [0]:
#Retirando os caracteres especiais
import re
def corrige_colunas(col_name):
    return re.sub(r"[/| ]", "", col_name)

In [0]:
df.columns = [corrige_colunas(col) for col in df.columns]

In [7]:
#Verificando os dados de coluna
cons = pd.DataFrame({'colunas' : df.columns,
                    'tipo': df.dtypes,
                    'nulos' : df.isna().sum(),
                    'quantidade' : df.shape[0],
                    'unicos': df.nunique()})
cons['percentual'] = round(cons['nulos'] / cons['quantidade'],2)

cons

,colunas,tipo,nulos,quantidade,unicos,percentual
ProvinceState,ProvinceState,object,23125,33125,80,0.7
CountryRegion,CountryRegion,object,0,33125,188,0.0
Lat,Lat,float64,0,33125,256,0.0
Long,Long,float64,0,33125,259,0.0
Date,Date,datetime64[ns],0,33125,125,0.0
Confirmed,Confirmed,int64,0,33125,5346,0.0
Deaths,Deaths,int64,0,33125,1750,0.0
Recovered,Recovered,int64,0,33125,3217,0.0


Observando as colunas percebemos que apenas a coluna Province/State possui 70% dos seus valores nulos.


In [8]:
df.head()

,ProvinceState,CountryRegion,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0
1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0,0
2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0,0
3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0,0
4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0,0


O dataset possui os números de casos confirmados, mortos e recuperados. 

**Casos Ativos**

Para nossa análise os necessitamos saber o total de casos ativos, para isso subtraimos dos casos confirmados o número de mortes e casos recuperados.

In [0]:
#Casos ativos
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']

**Casos nulos**

Nesse DataSet temos alguns valores nulos, então trataremos esses dados nessa etapa. A forma com que tratamos esses dados é feita de acordo com o método escolhido pelo cientista da análise. 

Nesta análise os valores nulos na coluna *ProvinceState* serão substituídos por um espaço. Já os valores nulos nas colunas *Confirmed, Deaths, Recovered e Ativos* substituímos os valores nulos por 0 (zero).

In [0]:
df[['ProvinceState']] = df[['ProvinceState']].fillna('')
df[['Confirmed','Deaths','Recovered','Active']] = df[['Confirmed','Deaths','Recovered','Active']].fillna('0')

Verificando como ficou nosso DataFrame.

In [13]:
df.head()

,ProvinceState,CountryRegion,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0,0
1,,Albania,41.1533,20.1683,2020-01-22,0,0,0,0
2,,Algeria,28.0339,1.6596,2020-01-22,0,0,0,0
3,,Andorra,42.5063,1.5218,2020-01-22,0,0,0,0
4,,Angola,-11.2027,17.8739,2020-01-22,0,0,0,0


**Analisando os dados temporais**

Agora analisaremos as informações sobre as séries temporais que temos disponíveis.

In [14]:
df.Date.describe()

count                   33125
unique                    125
top       2020-04-09 00:00:00
freq                      265
first     2020-01-22 00:00:00
last      2020-05-25 00:00:00
Name: Date, dtype: object

Temos nesse DataFrame um total de 33125 dados temporais, sendo destes 125 dados únicos. 

A primeira data reportada nesse dataframe é 22 de janeiro de 2020 e a última dia 25 de maio de 2020.

**Agrupando os dados**

Para facilitar as análises os dados serão agrupados.

* Agruparemos os dados por data e região.

In [0]:
df_grouped = df.groupby(['Date', 'CountryRegion'])['Confirmed','Deaths','Recovered','Active'].sum().reset_index()

* Os dados serão organizados em ordem descrescente do local com maior numero de casos, para o menor.

In [16]:
df_grouped.sort_values(by='Confirmed', ascending=False)

,Date,CountryRegion,Confirmed,Deaths,Recovered,Active
23486,2020-05-25,US,1662302,98220,379157,1184925
23298,2020-05-24,US,1643246,97720,366736,1178790
23110,2020-05-23,US,1622612,97087,361239,1164286
22922,2020-05-22,US,1600937,95979,350135,1154823
22734,2020-05-21,US,1577147,94702,298418,1184027
...,...,...,...,...,...,...
6031,2020-02-23,Belarus,0,0,0,0
6033,2020-02-23,Belize,0,0,0,0
6034,2020-02-23,Benin,0,0,0,0
6035,2020-02-23,Bhutan,0,0,0,0


* Iremos agrupar os dados por região e ordenar pela ordem decrescente

In [0]:
df_region = df.groupby('CountryRegion')['Confirmed','Deaths','Recovered','Active'].sum().reset_index()

In [18]:
df_region.sort_values(by='Confirmed', ascending=False)

,CountryRegion,Confirmed,Deaths,Recovered,Active
174,US,55417854,3123056,8043776,44251022
85,Italy,11665792,1535321,4210157,5920314
158,Spain,11654432,1274893,5619895,4759644
178,United Kingdom,8883938,1291876,42668,7549394
66,Germany,8865057,311294,5940931,2612832
...,...,...,...,...,...
38,Comoros,557,20,112,425
103,MS Zaandam,503,110,0,393
131,Papua New Guinea,351,0,176,175
184,Western Sahara,298,0,180,118


* Agrupando o DataFrame por data a quantidade de casos recuperados, mortos e ativos. 

In [0]:
df_temp = df.groupby('Date')['Confirmed','Deaths','Recovered','Active'].sum().reset_index()

Reeorganizando o Dataframe por data de acordo com a quantidade de casos recuperados, mortos e ativos.

In [0]:
df_temp = df_temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')

In [21]:
df_temp.head(5)

,Date,Case,Count
0,2020-01-22,Recovered,28
1,2020-01-23,Recovered,30
2,2020-01-24,Recovered,35
3,2020-01-25,Recovered,38
4,2020-01-26,Recovered,51


**Análises dos dados**


Vamos analisar os dados do Brasil:

* Primeiro passo: investigar as variáveis que temos a nossa disposição. Temos as informaçẽos disponíveis por Estado? 

In [0]:
df.loc[df.countryregion == 'Brazil']

No Brasil não temos a informação de casos de COVID-19 a nível de estado. Verificaremos algumas outras varíaveis para verificar o avanço do COVID-19 no país.